In [51]:
# -*- coding: utf-8 -*-
import matplotlib.pyplot as plt    # import matplotlib
import numpy as np                 # import numpy
import scipy as sp                 # import scipy
import scipy.stats
import math                        # import basic math functions
import random                      # import basic random number generator functions
import pandas as pd
import jieba as jb
import itertools
from itertools import combinations, permutations, repeat

In [52]:
df = pd.read_html('file:///Users/zimine/Desktop/Generator/words17k_CVVX_T.html', header=0)[0]
#Add [0] in the end because read_html return list of DataFrames

In [115]:
#---------------------------Unicode Problem---------------------------
#df['Words'][0]
#'\ufeff的'
#The Unicode character U+FEFF is the byte order mark, or BOM, and is used to tell the difference between big- and little-endian UTF-16 encoding. If you decode the web page using the right codec, Python will remove it for you. Examples:

In [53]:
#Reset the first element to remove the unicode character
df.at[0, 'Words'] = '的'

#Extract rows that contain single character from df
df1 = df[df['Words'].apply(lambda x: len(x) == 1)]

#Select rows that contain multiple characters and word(s) from df
df2 = df[df['Words'].apply(lambda x: len(x) > 1)]

In [54]:
#Extract index in df2
index = df2.index.values

#Convert index array to list 
index1 = index.tolist()

In [55]:
#Extract rows with multiple characters from df2 and save its indecies in index2 
df3 = pd.DataFrame() #initialize a dataframe
index2 = list()
index3 = list()
for i in index1:
    if df2['Words'][i][1] != ',':
        index2.append(i)

for i in index1:
    if df2['Words'][i][1] == ',':
        index3.append(i)

In [41]:
#-------------------------1st method--------------------------
#Extract rows with multiple characters from df
df3 = df.iloc[index3]

#Combine df1 with df3
frames = [df1, df3]
result = pd.concat(frames, axis=0, join='inner')

In [56]:
#-------------------------2nd method--------------------------
#Remove rows with multiple words from df
df4 = df.drop(index=index2)
df4

,Words,HomoDensity,Num-Neighbors,Freq-Neighbors
0,的,1,9,499618
1,我,1,261,4885412
2,"你, 妳, 拟",3,261,5241027
3,"是, 事, 试, 室, 式, 市, 士, 世, 视, 誓, 氏, 势, 嗜, 示, 逝, 饰...",29,306,6154407
4,"了, 瞭, 蓼",3,261,5622715
5,"不, 部, 步, 布, 簿, 钚, 怖, 钸, 咘, 埠, 鈈, 歩",12,304,6472516
6,"他, 她, 它, 塌, 祂, 牠, 铊",7,277,2553481
7,"在, 再, 载",3,305,6686024
9,"好, 号, 耗, 浩, 镐, 昊, 皓, 晧",8,304,6825829
10,"有, 友, 黝, 酉, 铕, 莠",6,262,6324260


In [57]:
#-------------------------2nd step--------------------------
#Import SUBTLEX-CH-CHR
df_chr = pd.read_csv("/Users/zimine/Documents/MPhil/Stimuli/Data/SUBTLEX-CH-CHR.csv")

In [58]:
#Creat two columns to df_chr and set all values to 0
df_chr['HomoDensity']= 0
df_chr['Num-Neighbors']= 0

#Get the index values of df_chr
index_chr = df_chr.index.values

#Convert the index array to list
index_chr_new = index_chr.tolist()

#Search in df4 to look for every character in df_chr
#Record the homodensity of every character found in df4
#Fill the homodensity of every character in df_chr
for i in index_chr_new: 
    if any(df4['Words'].str.contains(df_chr['Character'][i])) == True:
        a = int(df4.index.values[df4['Words'].str.contains(df_chr['Character'][i]) == True])
        b = df4.at[a,'HomoDensity']
        df_chr.at[i, 'HomoDensity'] = b
    else:
        df_chr.at[i, 'HomoDensity'] = -1

#df_chr

In [59]:
#-------------------------3rd step--------------------------
#Search in df4 to look for every character in df_chr
#Record the Num-Neighbours of every character found in df4
#Fill the Num-Neighbours of every character in df_chr
for i in index_chr_new: 
    if any(df4['Words'].str.contains(df_chr['Character'][i])) == True:
        a = int(df4.index.values[df4['Words'].str.contains(df_chr['Character'][i]) == True])
        b = df4.at[a,'Num-Neighbors']
        df_chr.at[i, 'Num-Neighbors'] = b
    else:
        df_chr.at[i, 'Num-Neighbors'] = -1
        
#df_chr

In [60]:
#Extract all characters do not appear in df4
df_miss = df_chr.loc[df_chr['HomoDensity'] == -1]
index_miss = df_miss.index.values
index_miss_list = index_miss.tolist()
len(index_miss_list)

1062

In [61]:
for k in index_miss_list:
    df_chr.drop(k, inplace=True)
#df_chr

In [64]:
#Save the dataframe to csv and return to original path
df_chr.to_csv('CLP_pho', index=False)

In [63]:
df_chr

,Character,CHR/million,logCHR,logCHR-CD,HomoDensity,Num-Neighbors
0,我,43956.70,6.3137,3.7953,1,261
1,的,37618.23,6.2460,3.7954,1,9
2,你,30845.37,6.1598,3.7953,3,261
3,是,25022.86,6.0690,3.7954,29,306
4,了,21220.47,5.9974,3.7954,3,261
5,不,19313.70,5.9565,3.7954,12,304
6,们,14981.59,5.8462,3.7954,1,10
7,这,13531.75,5.8020,3.7954,2,306
8,一,12895.86,5.7811,3.7952,10,278
9,他,12050.89,5.7516,3.7953,7,277


In [ ]:
#-------------------------4th step--------------------------
#Check whether a character appears in df4
a = df4.index.values[df4['Words'].str.contains(df_chr['Character'][4]) == True]

In [61]:
#Check whether a character appears in df4
df4.index.values[df4['Words'].str.contains('萄')]
df4['Words'][]

array([ 4824,  5894,  8275,  9615, 11004, 12380, 12882, 19855, 20987,
       22025, 27841, 30581, 44589])

In [65]:
#Check whether a character appears in original df
df.index.values[df['Words'].str.contains('萄')]
df['Words'][]

'葡萄园'

In [ ]:
for i in index_pho_new: 
    if any(df1['Character-1'].str.contains(df_pho['Character'][i])) == True:
        a = df1.index.values[df1['Character-1'].str.contains(df_pho['Character'][i]) == True]
        if len(a) == 1:
            b = df_pho.at[i,'HomoDensity']
            df1.at[a[0],'HD-1'] = b
        else: 
            for k in a:
                m = df_pho.at[i,'HomoDensity']
                df1.at[k,'HD-1'] = m
    else:
        df1.at[i, 'HD-1'] = -1